# Mounting drive
For this to work, we made a dataset folder and placed the CSVs into that folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import numpy as np
import pandas as pd
import csv

# Importing data from drive

In [ ]:
# Import data
path = "/content/drive/MyDrive/dataset/T1046_flow_raw_Gp11_Low Wai Qun.csv"
wq = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1190_flow_raw_Gp11_Tay Wei Jie.csv"
wj = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1048_flow_raw_Gp11_Quah Kian Yang.csv"
ky = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1053_flow_raw_Gp11_Koh Cheng Kiat.csv"
ck = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1548.001_flow_raw_Gp11_Ng Wei Liang.csv"
wl = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1210_flow_raw_Gp11_Lum Zheng Jie.csv"
zj = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/Packetbeat_Full_raw_Grp12_fixed.csv"
grp12 = pd.read_csv(path)

path = "/content/drive/MyDrive/dataset/T1595-T1570-T1020_Packetbeat_raw_Gp16_SimYewSiangMerrill-SimKaiChing-RachelWongSiHui-YeoHanJordan_fixed.csv"
grp16 = pd.read_csv(path)

# Fixing columns
One of our dataset had their columns values placed one column apart. Below fixes it.

### Fix Group 11 dataset for inconsistent columns

In [ ]:
# Fix columns
rows_to_shift = zj.loc[zj['type'].isna()].index
zj.iloc[rows_to_shift,30:] = zj.iloc[rows_to_shift,30:].shift(1,axis=1)
zj.loc[zj['host.id'].isna(), 'host.id'] = '-'

### Fix Group 12 and 16 dataset for inconsistent columns

In [ ]:
dataset_path = '/content/drive/MyDrive/dataset/' # Ensure that the dataset folder is in your Google Drive's root folder (can be a shortcut).

def log_fix(log_filename):
  final_data = []
  # opening the CSV file
  with open(dataset_path + log_filename, mode ='r')as file:
    # reading the CSV file
    csvFile = csv.reader(file)
  
    # displaying the contents of the CSV file
    for n, lines in enumerate(csvFile):
      if n == 0:
        final_data.append(['@timestamp', '_id', '_index', '_score', 'agent.ephemeral_id', 'agent.hostname', 'agent.id', 'agent.name', 'agent.type', 'agent.version', 'bytes_in', 'bytes_out', 'destination.bytes', 'destination.ip', 'destination.mac', 'destination.packets', 'destination.port', 'ecs.version', 'event.action', 'event.category', 'event.dataset', 'event.duration', 'event.end', 'event.kind', 'event.start', 'event.type', 'flow.final', 'flow.id', 'host.architecture', 'host.containerized', 'host.hostname', 'host.id', 'host.ip', 'host.mac', 'host.name', 'host.os.codename', 'host.os.family', 'host.os.kernel', 'host.os.name', 'host.os.name.text', 'host.os.platform', 'host.os.type', 'host.os.version', 'network.bytes', 'network.community_id', 'network.packets', 'network.transport', 'network.type', 'source.bytes', 'source.ip', 'source.mac', 'source.packets', 'source.port', 'type'])
      else:
        if lines[53] == "flow":
          if lines[47] == "ipv4":
            final_data.append(lines[:54])
        elif lines[64] == "icmp":
          pass
        elif lines[64] == "flow":
          if lines[12] != '-' and lines[13] != '-':
            for i in range(4):
              lines.pop(45)
            lines.pop(47)
            for i in range(3):
              lines.pop(50)
            lines.pop(55)
            lines.pop(43)
            lines.pop(43)
            final_data.append(lines[:54])
          else:
            if lines[54] == 'ipv4':
              lines.pop(12)
              lines.pop(12)
              for i in range(4):
                lines.pop(43)
              lines.pop(45)
              lines.pop(48)
              lines.pop(48)
              lines.pop(48)
              lines.pop(53)
              final_data.append(lines[:54])
        elif lines[77] == "flow":
          if lines[64] == "ipv4":
            lines.pop(12)
            lines.pop(12)
            lines.pop(12)
            lines.pop(18)
            for i in range(11):
              lines.pop(43)
            lines.pop(45)
            lines.pop(46)
            for i in range(5):
              lines.pop(48)
            lines.pop(48)
            lines.pop(53)
            final_data.append(lines[:54])
        elif lines[79] == "flow":
          if lines[66] == "ipv4":
            for i in range(3):
              lines.pop(12)
            lines.pop(18)
            for i in range(13):
              lines.pop(43)
            lines.pop(45)
            lines.pop(46)
            for i in range(6):
              lines.pop(48)
            lines.pop(53)
            final_data.append(lines[:54])
        elif lines[80] == "flow":
          if lines[66] == "ipv4":
            for i in range(3):
              lines.pop(12)
            lines.pop(13)
            for i in range(13):
              lines.pop(43)
            lines.pop(45)
            lines.pop(46)
            for i in range(7):
              lines.pop(48)
            lines.pop(53)
            final_data.append(lines[:54])
  with open(dataset_path + log_filename[:-4] + '_fixed.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows(final_data) 

In [ ]:
log_fix("Packetbeat_Full_raw_Grp12.csv")
log_fix("T1595-T1570-T1020_Packetbeat_raw_Gp16_SimYewSiangMerrill-SimKaiChing-RachelWongSiHui-YeoHanJordan.csv")

# Labeling of data
Label all port scan flows as attack, others as 0

In [ ]:
def data_cleaning(df):
  cols_to_convert = ['source.bytes', 'destination.bytes', 'bytes_in', 'bytes_out', 'network.bytes']

  for i, row in df.iterrows():
    for j in cols_to_convert:
      row[j] = row[j].replace(",", "")
      
      if row[j][-2:] == 'MB':
        df.at[i , j] = str(int(round(float(row[j][:-2]) * 1048576,0)))
      elif row[j][-2:] == 'KB':
        df.at[i , j] = str(int(round(float(row[j][:-2]) * 1024,0)))
      elif row[j][-1:] == 'B':
        df.at[i , j] = str(float(row[j][:-1]))

In [ ]:
# Label data
wq['attack'] = 0
wq.loc[(wq['source.ip'] == "10.10.10.10") & (wq['destination.ip'] == "10.10.10.30"), 'attack'] = 1

wj['attack'] = 0
ck['attack'] = 0
ky['attack'] = 0
wl['attack'] = 0

zj['attack'] = 0
zj_filter = (zj['source.ip'] == '10.10.10.10') & ((zj['destination.ip'] == '10.10.10.10') | (zj['destination.ip'] == '10.10.10.20') | (zj['destination.ip'] == '10.10.10.30')) & ((zj['destination.port'] != '3,306') & (zj['destination.port'] != '445'))
zj.loc[zj_filter, 'attack'] = 1

# Group 12
data_cleaning(grp12)
grp12['attack'] = 0
grp12.loc[(grp12['source.port'] == 58145), 'attack'] = 1

# Group 16
data_cleaning(grp16)
grp16['attack'] = 0

for i, row in grp16.iterrows():
  if float(row['network.bytes']) <= 1024 and row['source.ip'] == "192.168.0.4" and row['destination.ip'] == "192.168.0.3":
    grp16.at[i , 'attack'] = '1'

# Combining datasets into one

In [ ]:
frames = [wq, wj, ky, ck, wl, zj, grp12, grp16]

dataset = pd.concat(frames)

In [ ]:
dataset.shape

(56641, 55)

# Cleaning of data
- Drop duplicate columns
- Replace '-' with 0
- Remove ','
- Convert to float64
- Filter out selected features

In [ ]:
# Drop duplicates
dataset.drop_duplicates(subset='_id', keep="first")

# Replace - with 0
dataset.replace("-", 0, inplace = True)

# Convert to float64
dataset.replace(',','', regex=True, inplace=True)

dataset = dataset.astype({
    'bytes_in': 'float64',
    'bytes_out': 'float64',
    'destination.bytes': 'float64', 
    'source.bytes': 'float64',
    'network.bytes': 'float64',
    'event.duration': 'float64',
    'source.ip': 'str',
    'destination.ip': 'str',
    'source.port': 'float64',
    'destination.port': 'float64',
    'attack': 'float64'  
})

# Filter out features
selected_columns = ['bytes_in', 'bytes_out', 'destination.bytes', 'source.bytes', 'network.bytes', 'event.duration', 'source.ip', 'destination.ip', 'source.port', 'destination.port', 'attack']
filtered_dataset = dataset[selected_columns]

# Output cleaned data

In [ ]:
save_path = "/content/drive/MyDrive/dataset/all_data.csv"
filtered_dataset.to_csv(save_path, sep=',', index=False)